## import Libraries

In [75]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt;

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import DecomposeResult, seasonal_decompose
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer

from tensorflow import keras
from tensorflow.keras import layers

import plotly.graph_objs as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

## Data Preprocessing

In [76]:
# load dataset
df = pd.read_csv('penjualan_mobil.csv')

In [77]:
# menampilkan dataset
df.head()

,Bulan,Penjualan
0,2011-01,27619
1,2011-02,25532
2,2011-03,32275
3,2011-04,21128
4,2011-05,19554


In [78]:
# cek nilai null
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Bulan      149 non-null    object
 1   Penjualan  149 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


In [79]:
# Konversi kolom Bulan kedalam format date.
df.index = pd.to_datetime(df.Bulan, format='%Y-%m')

In [80]:
df = df.resample("M").sum()

In [81]:
df.head()

,Penjualan
Bulan,
2011-01-31,27619
2011-02-28,25532
2011-03-31,32275
2011-04-30,21128
2011-05-31,19554


In [82]:
# Menampilkan grafik penjualan mobil di Indonesia selama tahun 2011 - 2023.
trace0= go.Scatter(
        x = df.index,
        y = df.Penjualan,
        mode ='lines',
        name = 'Penjualan'
)

data=[trace0]
layout = go.Layout(title=f'<b>Penjualan Mobil di Indonesia Tahun 2011 - 2023</b>', title_x=0.5, xaxis_title='Bulan', yaxis_title='Penjualan')

fig = go.Figure(data=data,layout=layout)
fig.show()

In [83]:
def plot_seasonal_decompose(result:DecomposeResult, dates:pd.Series=None, title:str="Seasonal Decomposition"):
    x_values = dates if dates is not None else np.arange(len(result.observed))
    return (
        make_subplots(
            rows=4,
            cols=1,
            subplot_titles=["Observed", "Trend", "Seasonal", "Residuals"],
        )
        .add_trace(
            go.Scatter(x=x_values, y=result.observed, mode="lines", name='Observed'),
            row=1,
            col=1,
        )
        .add_trace(
            go.Scatter(x=x_values, y=result.trend, mode="lines", name='Trend'),
            row=2,
            col=1,
        )
        .add_trace(
            go.Scatter(x=x_values, y=result.seasonal, mode="lines", name='Seasonal'),
            row=3,
            col=1,
        )
        .add_trace(
            go.Scatter(x=x_values, y=result.resid, mode="lines", name='Residual'),
            row=4,
            col=1,
        )
        .update_layout(
            height=900, title=f'<b>{title}</b>', margin={'t':100}, title_x=0.5, showlegend=False
        )
    )

In [84]:
# Menampilkan grafik seasonal decompose.
decomposition = seasonal_decompose(df['Penjualan'], model='additive', period=12)
fig = plot_seasonal_decompose(decomposition, dates=df.index)
fig.show()

In [85]:
# Membagi data latih dan data uji.
latih = df[:-6]
uji = df[-6:]

In [86]:
# Menampilkan tabel data latih.
latih.head()

,Penjualan
Bulan,
2011-01-31,27619
2011-02-28,25532
2011-03-31,32275
2011-04-30,21128
2011-05-31,19554


In [87]:
# Menampilkan tabel data uji.
uji.head()

,Penjualan
Bulan,
2022-12-31,28128
2023-01-31,28970
2023-02-28,27336
2023-03-31,29471
2023-04-30,21518


In [88]:
# Membentuk lags.
def lag(df, n):
    X, y = [], []
    for i in range(len(df) - n):
        X.append(df[i:i+n])
        y.append(df[n+i])

    return np.array(X), np.array(y), np.array(y[-n:]).reshape(1,n)

In [89]:
# Memanggil fungsi lag.
X, y, batch_terakhir = lag(latih.Penjualan.values, 12)

In [90]:
X

array([[27619, 25532, 32275, ..., 31109, 15195, 26076],
       [25532, 32275, 21128, ..., 15195, 26076, 29189],
       [32275, 21128, 19554, ..., 26076, 29189, 33558],
       ...,
       [34046, 21730, 33450, ..., 27290, 29326, 30844],
       [21730, 33450, 32707, ..., 29326, 30844, 33449],
       [33450, 32707, 22886, ..., 30844, 33449, 33740]])

In [91]:
y

array([29189, 33558, 33306, 34264, 34737, 37176, 36353, 25848, 33705,
       35855, 36996, 34427, 35923, 35318, 32726, 39668, 36282, 35125,
       39210, 24899, 40235, 39246, 40781, 34819, 35886, 38631, 38959,
       39323, 31440, 39107, 28757, 30273, 29247, 31538, 28835, 27123,
       27166, 26743, 31801, 30053, 23223, 23967, 13495, 31991, 32405,
       29826, 29607, 21541, 24859, 25447, 29995, 31059, 31662, 31519,
       24718, 38803, 36994, 34116, 40510, 31888, 32377, 36147, 38656,
       33328, 32806, 21974, 31621, 34690, 27251, 29979, 28797, 23706,
       25405, 27665, 31424, 29360, 28950, 18642, 34984, 31149, 29821,
       36119, 31981, 26661, 25081, 23443, 28732, 29408, 29103, 18541,
       29362, 28929, 31831, 30944, 28970, 27453, 24119, 25053, 26191,
        2053,   695,  3705,  7224,  8673, 13150, 16345, 15361, 18687,
       16033, 15144, 26258, 23301, 18167, 23271, 21763, 29898, 34046,
       21730, 33450, 32707, 22886, 24865, 33344, 27779, 13297, 27290,
       29326, 30844,

## Membuat Model

In [92]:
# Membuat fungsi untuk model Multi-Layer Perceptron.
def mlp(input_, y_, epochs):
    model = Sequential()
    model.add(Dense(10, activation='relu', input_dim=input_.shape[1]))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(1))

#   opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer='adam', loss='mape')
    print(model.fit(input_, y_, epochs=epochs))
    return model

## Memanggil fungsi model.

In [93]:
# Menjalankan model dengan epoch = 300.
model = mlp(X, y, 300)

Epoch 1/300
5/5 [==============================] - 1s 3ms/step - loss: 71.9276
Epoch 2/300
5/5 [==============================] - 0s 3ms/step - loss: 60.2719
Epoch 3/300
5/5 [==============================] - 0s 3ms/step - loss: 53.9850
Epoch 4/300
5/5 [==============================] - 0s 4ms/step - loss: 54.1644
Epoch 5/300
5/5 [==============================] - 0s 7ms/step - loss: 53.8527
Epoch 6/300
5/5 [==============================] - 0s 3ms/step - loss: 51.6733
Epoch 7/300
5/5 [==============================] - 0s 3ms/step - loss: 50.8871
Epoch 8/300
5/5 [==============================] - 0s 3ms/step - loss: 49.7407
Epoch 9/300
5/5 [==============================] - 0s 3ms/step - loss: 49.0731
Epoch 10/300
5/5 [==============================] - 0s 3ms/step - loss: 48.3795
Epoch 11/300
5/5 [==============================] - 0s 3ms/step - loss: 47.9400
Epoch 12/300
5/5 [==============================] - 0s 3ms/step - loss: 47.4076
Epoch 13/300
5/5 [==============================]

In [94]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                130       
                                                                 
 dense_7 (Dense)             (None, 15)                165       
                                                                 
 dense_8 (Dense)             (None, 1)                 16        
                                                                 
Total params: 311 (1.21 KB)
Trainable params: 311 (1.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Prediksi Penjualan

In [95]:
# Membuat fungsi prediksi penjualan mobil
def fungsi_prediksi(model, batch_terakhir, n):
    in_value = batch_terakhir.copy()
    preds = []
    for i in range(n):
        p = model.predict(in_value)
        preds.append(p.ravel())
        in_value = np.append(in_value, p)[1:].reshape(batch_terakhir.shape)
    return np.array(preds).ravel()

In [96]:
# Memanggil fungsi prediksi
prediksi = fungsi_prediksi(model, batch_terakhir, 6)

1/1 [==============================] - 0s 64ms/step


1/1 [==============================] - 0s 22ms/step


In [97]:
# Hasil prediksi yang diperoleh:
prediksi

array([26807.328, 28339.662, 28963.541, 24260.715, 22023.006, 26129.934],
      dtype=float32)

In [98]:
# Menyimpan nilai prediksi ke dalam tabel uji pada kolom baru "Prediksi_Penjualan".
uji['Prediksi_Penjualan'] = prediksi

In [99]:
# Menampilkan tabel uji setelah penambahan.
uji

,Penjualan,Prediksi_Penjualan
Bulan,,
2022-12-31,28128,26807.328125
2023-01-31,28970,28339.662109
2023-02-28,27336,28963.541016
2023-03-31,29471,24260.714844
2023-04-30,21518,22023.005859
2023-05-31,28178,26129.933594


In [100]:
# Menampilkan grafik perbedaan antara Penjualan asli dan Penjualan hasil prediksi.
trace0= go.Scatter(
        x = uji.index,
        y = uji.Penjualan,
        mode ='lines+markers',
        name = 'Penjualan'
)

trace1 = go.Scatter(
        x = uji.index,
        y = uji.Prediksi_Penjualan,
        mode ='lines+markers',
        name = 'Prediksi_Penjualan'
)

data=[trace0,trace1]
layout = go.Layout(title=f'<b>Perbandingan Penjualan Asli dan Prediksi Penjualan</b>', title_x=0.5, legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
), xaxis_title='Bulan', yaxis_title='Penjualan')

fig = go.Figure(data=data,layout=layout)
fig.show()

## Nilai Error

In [101]:
# Membuat fungsi untuk menghitung nilai error.
def hitung_error(df,kolom_1,kolom_2):
    data = df.copy()
    list_ka = []
    list_pka = []
    for i in range(len(data)):
        ka = abs(data[kolom_1][i]-data[kolom_2][i])
        list_ka.append(ka)
        pka = abs((data[kolom_1][i]-data[kolom_2][i])/data[kolom_1][i])*100
        list_pka.append(pka)
    data['Kesalahan_Absolut'] = list_ka
    data['Persentase_Kesalahan_Absolut'] = list_pka
    return data

In [102]:
# Memanggil fungsi nilai error.
kolom_1 = 'Penjualan'
kolom_2 = 'Prediksi_Penjualan'
df_new = hitung_error(uji,kolom_1,kolom_2)

In [103]:
# Menampilkan tabel beserta hasil dari nilai errornya.
df_new

,Penjualan,Prediksi_Penjualan,Kesalahan_Absolut,Persentase_Kesalahan_Absolut
Bulan,,,,
2022-12-31,28128,26807.328125,1320.671875,4.695221
2023-01-31,28970,28339.662109,630.337891,2.175830
2023-02-28,27336,28963.541016,1627.541016,5.953837
2023-03-31,29471,24260.714844,5210.285156,17.679363
2023-04-30,21518,22023.005859,505.005859,2.346900
2023-05-31,28178,26129.933594,2048.066406,7.268317


In [104]:
# Menghitung Nilai MAE dan MAPE.
mae = sum(df_new['Kesalahan_Absolut'])/len(df_new)
mape = sum(df_new['Persentase_Kesalahan_Absolut'])/len(df_new)
print('Nilai MAE : ', mae)
print('Nilai MAPE: ', mape, '%')

Nilai MAE :  1890.3180338541667
Nilai MAPE:  6.686578122744602 %
